In [2]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import pandas as pd
import copy
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from scipy import stats
from sklearn.metrics import confusion_matrix
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score

In [8]:
#加载数据 预处理
data = pd.read_csv("../data/gene_version2020/generate/ExperimentalDatasets/MM/MM-BP.csv",index_col=0)
label = data["longevity influence"]
#删除无用列
data = data.drop(["longevity influence"],axis=1)

#初始化基因本体的DAG
DAG = pd.read_csv("../data/gene_version2020/generate/GOPath/MM/MM-BP.csv",index_col=0)

#过滤低纬度特征
for c in data.columns:
    data[c] = data[c].astype('int')
    if(data[c].sum() < 3):
        data.drop(c,axis=1,inplace=True)
DAG = DAG.loc[data.columns,data.columns]

#计算相关性
data["label"] = label
Relevance = pd.Series(0.0,index=DAG.columns)
for name in DAG.columns:
    Relevance[name] = getRelevance(data,name)
data = data.drop(["label"],axis=1)

In [17]:
data

,GO:0016239,GO:0055114,GO:0071230,GO:0015031,GO:0000209,GO:0006511,GO:0008340,GO:0016567,GO:0043161,GO:0000122,...,GO:0006937,GO:0007626,GO:0090325,GO:0043058,GO:0048557,GO:0007608,GO:0006006,GO:0007218,GO:0010035,GO:0006869
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2164,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2167,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
def getRelevance(data,name):#计算相关性
    a = data[data[name] == 1]["label"].mean()
    b = data[data[name] == 0]["label"].mean()
    
    c = 1-a
    d = 1-b
    return pow((a-b),2)+pow((c-d),2)

def GM_score(x,y): #计算GM 传入numpy
    cm1 = confusion_matrix(x,y)
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    return sensitivity1,specificity1

def getAncestors(name):
    global relativeList
    for relative in DAG.columns:
        if(DAG.loc[relative,name] == 1):
            relativeList.append(relative)
            getAncestors(relative)

def getDescendants(name):
    global relativeList
    for relative in DAG.columns:
        if(DAG.loc[name,relative] == 1):
            relativeList.append(relative)
            getAncestors(relative)

def HIP(x): #传入实例，返回保留特征name
    global relativeList
    dropSet = set()
    for name in DAG.columns:
        relativeList = []
        if(x[name] == 1):
            getAncestors(name)
        else:
            getDescendants(name)
        for removeName in set(relativeList):
            dropSet.add(removeName)
        
    return list(set(DAG.columns)-dropSet)

def MR(x):
    global pathList
    dropSet = set()
    returnList = []
    for name in DAG.columns:
        pathList = []
        if(x[name] == 1):#找到所有至根节点路径
            getPathsToRoot([],name)
            for path in pathList:
                #保留相关性最高特征，其它特征加入dropSet
                dropSet.update(Relevance[path].sort_values(ascending=False).index[1:])
                #returnList.append(Relevance[path].sort_values(ascending=False).index[0])
        else:
            getPathsToLeft([],name)
            for path in pathList:
                #保留相关性最高特征，其它特征加入dropSet
                dropSet.update(Relevance[path].sort_values(ascending=False).index[1:])
                #returnList.aend(Relevance[path].sort_values(ascending=False).index[0])
        #print(pathList)
    return list(set(DAG.columns)-dropSet)
    #return list(set(returnList))

def HIP_MR(x):
    global pathList
    returnSet = set()
    for name in DAG.columns:
        if(x[name] == 1):#找到所有至根节点路径 删除相关性小于等于当前节点的特征
            getPathsToRoot([],name)
            for path in pathList:
                #删除相关性小于等于当前节点的特征
                threshold = Relevance[name]
                temp = Relevance[Relevance > threshold][path].index.tolist()
                returnSet.update(set(temp.append(name)))
        else:
            getPathsToLeft([],name)
            for path in pathList:
                #删除相关性小于等于当前节点的特征
                threshold = Relevance[name]
                temp = Relevance[Relevance > threshold][path].index.tolist()
                returnSet.update(set(temp.append(name)))
    return list(returnSet)

def myknn(train,label,test):
    maxJcd = 0
    prediction = 0
    for i in range(train.shape[0]):
        temp = jaccard_score(train.iloc[i,:],test)
        if(temp > maxJcd):
            prediction = label[i]
            maxJcd = temp
    return prediction

def getPathsToRoot(path,name):#传入list 方便递归
    global pathList
    flag = 0
    path.append(name)
    for anc in DAG.columns:
        if(DAG.loc[anc,name] == 1):
            getPathsToRoot(copy.copy(path),anc)
            flag = 1
    if(flag == 0):
        pathList.append(path)
def getPathsToLeft(path,name):#传入list 方便递归
    global pathList
    flag = 0
    path.append(name)
    for dec in DAG.columns:
        if(DAG.loc[name,dec] == 1):
            flag = 1
            getPathsToLeft(copy.copy(path),dec)
    if(flag == 0):
        pathList.append(path)

In [12]:
pathList = []
relativeList = []
sensitivity = []
specificity = []
F1 = []
AUC = []

#10折交叉验证
kf = KFold(n_splits=10,shuffle=False)
for train_index ,test_index in kf.split(data):
    trainData = data.iloc[train_index,:]
    testData = data.iloc[test_index,:]
    Y_train = label.values[train_index] #用于训练模型
    Y_test = label.values[test_index]#用于交叉验证
    predicetList = []
    
    
    for i in range(testData.shape[0]):  #对每一个样本进行特征选择和预测
        global pathList
        global relativeList
        
        #selectFeatures = testData.columns #保留所有特征
        #selectFeatures = HIP(testData.iloc[i,:]) #HIP特征选择
        selectFeatures = MR(testData.iloc[i,:]) #MR特征选择
        #selectFeatures = HIP_MR(testData.iloc[i,:]) 
        
        #print(len(selectFeatures))
        X_train = trainData.loc[:,selectFeatures]
        X_test = testData.loc[:,selectFeatures].iloc[i,:]
        
        gnb = GaussianNB()
        predicetList.append(gnb.fit(X_train, Y_train).predict(X_test.values.reshape(1,len(selectFeatures)))[0])#把当前样本预测结果加入List
        #predicetList.append(myknn(X_train,Y_train,X_test))
    print(predicetList)
    print(Y_test)
    sensi,speci= GM_score(np.array(predicetList),Y_test)
    sensitivity.append(sensi)
    specificity.append(speci)
    F1.append(f1_score(np.array(predicetList),Y_test))
    AUC.append(roc_auc_score(np.array(predicetList),Y_test))

[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0 1 1 1 1 1 1 1 1 1 1 0 1]
[1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0]
[0 1 1 1 1 1 1 0 0 0 0 1 1]
[1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1]
[0 0 0 0 1 1 1 1 1 1 1 0 1]
[0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0]
[1 0 1 1 1 1 0 1 0 1 1 1 0]
[0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1]
[0 1 1 1 1 1 1 1 1 1 1 0 1]
[1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0]
[1 1 1 0 0 1 1 0 1 0 1 1 0]
[1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0]
[1 1 1 1 0 1 1 1 0 0 0 1 0]
[0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1]
[0 0 1 1 1 1 1 1 0 0 0 1 1]
[1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0]
[1 1 0 1 0 1 1 1 0 0 0 1 0]
[1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0]
[1 0 1 1 0 1 1 0 0 0 1 1 0]


In [13]:
a = np.nanmean(sensitivity)
b = np.nanmean(specificity)
print(a)
print(b)
print("GM")
print(math.sqrt(a*b))
print("f1")
print(np.nanmean(F1))
print("AUC")
print(np.nanmean(AUC))

0.6038095238095238
0.8114646464646466
GM
0.69997862950956
f1
0.7851387827858416
AUC
0.7076370851370851


In [55]:
relativeList = []
getAncestors("GO:0005634")
relativeList

['GO:0043231']

In [47]:
a = pd.Series([1,0,1,1,0])
b = pd.Series([0,1,0,0,1])
jaccard_score(a,b)

0.0